In [ ]:
import nltk
nltk.download('punkt')  # Make sure to download the punkt tokenizer data

def text_to_sentences(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        sentences = nltk.sent_tokenize(text)
    return sentences

file_path = '/content/big.txt'
sentences = text_to_sentences(file_path)

# for sentence in sentences:
#     print(sentence)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def save_selected_sentences(sentences, output_file):
    selected_sentences = sentences[2000:5000]
    with open(output_file, 'w', encoding='utf-8') as file:
        for sentence in selected_sentences:
            file.write(sentence + '\n')

output_file = 'source.txt'
save_selected_sentences(sentences, output_file)

In [ ]:
selected_sentences=sentences[2000:5000]

#Save in csv file IMPORTANT

In [ ]:
import csv

def replace_words_in_sentence(sentence, replacements, key):
    changes = []
    num_changes = 0
    for replacement in replacements.get(key, []):  # Check if the key exists in replacements
        if f" {key} " in sentence:
            sentence = sentence.replace(f" {key} ", f" {replacement} ")
            changes.append((key, replacement))
            num_changes += 1
        elif sentence.startswith(key + " ") and sentence[len(key)] == " ":
            sentence = sentence.replace(key + " ", replacement + " ", 1)
            changes.append((key, replacement))
            num_changes += 1
        elif sentence.endswith(" " + key) and sentence[-len(key) - 1] == " ":
            sentence = sentence[::-1].replace(key[::-1] + " ", replacement[::-1] + " ", 1)[::-1]
            changes.append((key, replacement))
            num_changes += 1
        elif sentence == key:
            sentence = replacement
            changes.append((key, replacement))
            num_changes += 1
    return sentence, num_changes, changes

def main():
    # Load dictionary
    with open('/content/wikipedia.txt', 'r') as dict_file:
        replacements = {}
        for line in dict_file:
            words = line.strip().split(':')
            key = words[0].strip()
            values = [v.strip() for v in words[1].split()]
            replacements[key] = values

    # Read source file
    with open('/content/source.txt', 'r') as source_file:
        source_text = source_file.read()

    # Split the source text into sentences
    sentences = source_text.split('.')

    # Perform replacements sentence by sentence
    replaced_sentences = []
    with open('replacements.csv', 'w', newline='') as csvfile:
        fieldnames = ['original_sentence', 'replaced_sentence', 'replacement', 'number_of_replacements', 'changed_words']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for sentence in sentences:
            for key in replacements.keys():
                replaced_sentence, num_changes, changes = replace_words_in_sentence(sentence, replacements, key)
                if num_changes > 0:  # Write to CSV only if there are changes
                    replaced_sentences.append(replaced_sentence)
                    writer.writerow({'original_sentence': sentence.strip(),
                                     'replaced_sentence': replaced_sentence.strip(),
                                     'replacement': 1,
                                     'number_of_replacements': num_changes,
                                     'changed_words': ', '.join([f'{change[0]}->{change[1]}' for change in changes])})
                    break  # Move to the next sentence once replacements are made
            else:
                replaced_sentences.append(sentence)  # If no replacements, keep the original sentence

    # Join the replaced sentences back into text
    replaced_text = '. '.join(replaced_sentences) + '.'

    # Write replaced text to a new file
    with open('replaced.txt', 'w') as replaced_file:
        replaced_file.write(replaced_text)

if __name__ == "__main__":
    main()


#level of difficulty


In [120]:
import pandas as pd
import random

# Read the CSV file into a DataFrame
df = pd.read_csv('/content/replacements.csv')

def random_number():
    return random.randint(1, 3)

# Add a new column 'level_of_difficulty' with random numbers
df['number_changes_wanted'] = [random_number() for _ in range(len(df))]


#Version3.0

In [ ]:
import pandas as pd
import numpy as np
import string
import random


In [ ]:
def case_variaties(word: str) -> list[str]:
    'Change the capitalisation of the first two letters'
    case_list = []
    case_list.append([word.lower(), word])
    case_list.append([word.upper(), word])
    case_list.append([word[:1].upper() + word[1:], word])
    case_list.append([word[:2].upper() + word[2:], word])
    return case_list

def enter_signs(word: str) -> list[str]:
    'add # and + at the end, because of fat fingers when hitting enter'
    enter_list = []
    enter_list.append([word + '#', word])
    enter_list.append([word + '+', word])
    return enter_list

def no_spaces(word: str) -> str:
    'eliminate all spaces from word'
    return [[word.replace(' ', ''), word]]

def swap_letter(word: str) -> list[str]:
    'swap every letter of a word pairwise'
    swap_list = []
    for idx, letter in enumerate(word):
        swap_list.append([word[:idx] + word[idx+1] + word[idx] + word[idx+2:], word])
        if idx + 2 == len(word):
            break
    return swap_list

def double_letter(word: str) -> list[str]:
    'double every letter in the word'
    double_letter_list = []
    for idx, letter in enumerate(word):
        double_letter_list.append([word[:idx] + letter + word[idx:], word])
    return double_letter_list

def one_out(word: str) -> list[str]:
    'Remove on every possible position one letter from the input word'
    one_out = []
    for idx in range(len(word)):
        one_out.append([word[:idx] + word[idx+1:], word])
    return one_out

def add_noise(word: str) -> list[str]:
    'add a random string with random length at the end of the word'
    noise_words = []
    for _ in range(6):
        noise_words.append([word + ''.join([random.choice(string.ascii_letters) for i in range(random.randint(1, 8))]), word])
    return noise_words


In [121]:

# Iterate over each row
for index, row in df.iterrows():
    changes_wanted = row['number_changes_wanted']
    current_replacements = row['number_of_replacements']

    if current_replacements < changes_wanted:
        # Choose randomly from functions and apply them to unchanged words
        original_sentence = row['original_sentence']
        changed_sentence = row['replaced_sentence']
        before_changed_words=row['changed_words'].split('->')
        before_changed_words=before_changed_words[0]
        changed_words = row['changed_words'].split('->')
        changed_words = changed_words[1]

        unchanged_words = [word for word in original_sentence.split() if word not in changed_words and len(word) > 3]

        while current_replacements < changes_wanted:
            word_to_change = random.choice(unchanged_words)
            function_to_apply = random.choice([case_variaties, enter_signs, no_spaces, swap_letter, double_letter, one_out, add_noise])
            modified_word = function_to_apply(word_to_change)[0][0]
            changed_sentence = changed_sentence.replace(word_to_change, modified_word, 1)
            current_replacements += 1
            # Initialize changed_words as a list if it's not already
            if isinstance(changed_words, str):
              changed_words = changed_words + "->" + before_changed_words
              changed_words = changed_words.split(', ')
            changed_words.append(f"{word_to_change}->{modified_word}")
        # Update DataFrame with modified sentence and number of replacements
        df.at[index, 'replaced_sentence'] = changed_sentence
        df.at[index, 'number_of_replacements'] = current_replacements
        # Update changed_words with newly replaced words


        df.at[index, 'changed_words'] = ', '.join(changed_words)
# Save the updated DataFrame back to the CSV file
df.to_csv('your_file.csv', index=False)
